In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",google_api_key="GOOGLE_API_KEY")

In [9]:
#state
class LLMState(TypedDict):
    title: str
    blog : str
    outline: str


#node
def generate_outline(state: LLMState) -> LLMState:
    #take query
    title = state['title']
    #prompt
    prompt = f"Generate a outline for a blog on this title {title}"
    #llm call
    response = model.invoke(prompt).content
    #update state
    state['outline'] = response
    return state

def generate_blog(state: LLMState) ->LLMState:
    #take outline
    outline = state['outline']
    title = state['title']
    #prompt
    prompt = f"Write a blog in 200 words in the following outline {outline} on title {title}"
    #invoke model
    response = model.invoke(prompt).content
    state['blog'] = response
    #update state
    return state
#edges

#compile

#invoke

In [12]:
graph = StateGraph(LLMState)

graph.add_node("generate_outline",generate_outline)
graph.add_node("generate_blog",generate_blog)

graph.add_edge(START,"generate_outline")
graph.add_edge("generate_outline","generate_blog")
graph.add_edge("generate_blog",END)

#compile

workflow = graph.compile()

In [13]:
prompt_input = {"title" : "AWS"}

In [14]:
response = workflow.invoke(prompt_input)

{'title': 'AWS',
 'blog': 'Here\'s a blog post about AWS, precisely 199 words, following your outline:\n\n## AWS Explained: Your Essential Guide to Amazon Web Services\n\nThe cloud has revolutionized technology, and at its heart is **Amazon Web Services (AWS)**. This comprehensive, on-demand cloud computing platform offers global services on a pay-as-you-go model. This guide introduces AWS, demystifying its core.\n\n**What Exactly is AWS?**\nAWS provides computing resources over the internet, replacing costly on-premise IT. Its global infrastructure ensures reliability and scalability. The "pay-as-you-go" model means you only pay for what you use, for cost-efficiency.\n\n**Why Choose AWS?**\nBusinesses choose AWS for rapid **scalability**, cost-effectiveness (eliminating hardware), and robust **reliability** and **security**. It also drives **agility** and **innovation** through its vast service portfolio.\n\n**A Glimpse into Key AWS Services**\nKey AWS services include **Amazon EC2** 